In [ ]:
import pandas as pd
import os
import io
import re
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2 import service_account
from googleapiclient.errors import HttpError
from PIL import Image
from PIL import Image


In [45]:
#Setup for directories
BASE_FOLDER = r"D:\ML-AI\AI-ML Project\Cat Gender Classification Dataset"

ALL_IMAGES_FOLDER = os.path.join(BASE_FOLDER, "All_Images")  # optional
MALE_FOLDER = os.path.join(BASE_FOLDER, "Male")
FEMALE_FOLDER = os.path.join(BASE_FOLDER, "Female")
INVALID_LINKS_FILE =  "invalid_links.txt"

os.makedirs(MALE_FOLDER, exist_ok=True)
os.makedirs(FEMALE_FOLDER, exist_ok=True)

In [46]:

#google api setup
SERVICE_ACCOUNT_FILE = "cobalt-mind-448907-s8-b3c4968a1078.json"

SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
drive_service = build('drive', 'v3', credentials=creds)
def extract_file_id(url):
    patterns = [r"open\?id=([^&]+)", r"d/([^/]+)/"]
    for p in patterns:
        m = re.search(p, url)
        if m:
            return m.group(1)
    return None



In [ ]:
male_counter = 1
female_counter = 1

invalid_links = []

for idx, row in df.iterrows():
    link = row['Image_Link']
    gender = str(row['Gender']).strip().capitalize()  # Male / Female
    
    file_id = extract_file_id(link)
    if not file_id:
        print(f"[SKIP] Invalid Drive link: {link}")
        invalid_links.append(link)
        continue
    
    try:
        # Download file from Drive
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
        
        fh.seek(0)
        
        # Set new filename
        ext = ".jpg"  # default
        if gender == "Male":
            filename = f"Cat_{str(male_counter).zfill(3)}_Male{ext}"
            folder = MALE_FOLDER
            male_counter += 1
        else:
            filename = f"Cat_{str(female_counter).zfill(3)}_Female{ext}"
            folder = FEMALE_FOLDER
            female_counter += 1
        
        save_path = os.path.join(folder, filename)
        
        # Save file locally
        with open(save_path, "wb") as f:
            f.write(fh.read())
        
        print(f"[DONE] {save_path}")
    
    except HttpError as e:
        print(f"[ERROR] Could not download file: {link} ({e})")
        invalid_links.append(link)
        continue
# Save invalid links to a text file
if invalid_links:
    with open(INVALID_LINKS_FILE, "w", encoding="utf-8") as f:
        for link in invalid_links:
            f.write(link + "\n")
    print(f"❌ Invalid links saved to {INVALID_LINKS_FILE}")

print("✅ All downloads complete!")

[DONE] D:\ML-AI\AI-ML Project\Cat Gender Classification Dataset\Male\Cat_001_Male.jpg
[DONE] D:\ML-AI\AI-ML Project\Cat Gender Classification Dataset\Female\Cat_001_Female.jpg
[DONE] D:\ML-AI\AI-ML Project\Cat Gender Classification Dataset\Male\Cat_002_Male.jpg
[DONE] D:\ML-AI\AI-ML Project\Cat Gender Classification Dataset\Female\Cat_002_Female.jpg
[DONE] D:\ML-AI\AI-ML Project\Cat Gender Classification Dataset\Female\Cat_003_Female.jpg
[DONE] D:\ML-AI\AI-ML Project\Cat Gender Classification Dataset\Male\Cat_003_Male.jpg
[DONE] D:\ML-AI\AI-ML Project\Cat Gender Classification Dataset\Male\Cat_004_Male.jpg
[DONE] D:\ML-AI\AI-ML Project\Cat Gender Classification Dataset\Male\Cat_005_Male.jpg
[DONE] D:\ML-AI\AI-ML Project\Cat Gender Classification Dataset\Male\Cat_006_Male.jpg
[DONE] D:\ML-AI\AI-ML Project\Cat Gender Classification Dataset\Female\Cat_004_Female.jpg
[DONE] D:\ML-AI\AI-ML Project\Cat Gender Classification Dataset\Male\Cat_007_Male.jpg
[DONE] D:\ML-AI\AI-ML Project\Cat Gend